In [ ]:
import numpy as np
import csv
from itertools import product, combinations
import tensorflow.compat.v1 as tf 
tf.compat.v1.disable_eager_execution()
import sys
sys.path.append("../")
import copy
import pandas as pd
from tensorflow.python.platform import flags
import time
from adf_data.census import census_data
from adf_data.credit import credit_data
from adf_data.compas import compas_data
from adf_data.default import default_data
from adf_data.bank import bank_data
from adf_data.heart import heart_data
from adf_model.tutorial_models import dnn
from adf_utils.utils_tf import model_prediction, model_argmax , layer_out
from adf_utils.config import census, credit, bank, compas, default, heart

FLAGS = flags.FLAGS
       
def m_instance( sample, sens_params, conf):
    index = []
    m_sample = []
    for sens in sens_params:
        index.append([i for i in range(conf.input_bounds[sens-1][0], conf.input_bounds[sens-1][1]+1)])
      
    for ind in list(product(*index)):     
        temp = sample.copy()
        for i in range(len(sens_params)):
            temp[0][sens_params[i]-1] = ind[i]
        m_sample.append(temp)
    return np.array(m_sample)
    
def clustering(probs,m_sample, sens_params):
    epsillon=0.0125
    cluster_dic = {}
    cluster_dic['Seed']=m_sample[0][0]

        #  to avoid k = Max + 1 
    for i in range(len(probs)):
    
        if probs[i] == 1.0:
            if (int( probs[i] / epsillon ) -1) not in cluster_dic.keys():
             
                cluster_dic[ (int( probs[i] / epsillon ) -1)] = [ [m_sample[i][0][j-1] for j in sens_params]]
           
            else:
                cluster_dic[ (int( probs[i] / epsillon ) -1)].append( [m_sample[i][0][j-1] for j in sens_params] )

                       
        elif int( probs[i] / epsillon ) not in cluster_dic.keys():
                cluster_dic[ int( probs[i] / epsillon )] = [ [m_sample[i][0][j-1] for j in sens_params] ]
           
        else:
                cluster_dic[ int( probs[i] / epsillon)].append( [m_sample[i][0][j-1] for j in sens_params] )

    return cluster_dic  

    
def pred_prob(sess, x, preds, m_sample, input_shape):
        #global probs
        probs = model_prediction(sess, x, preds, np.array(m_sample).reshape(len(m_sample),
                                                                            input_shape[1]))[:,1:2].reshape(len(m_sample))
        return probs        
        
def neuron_locator(samples, layer_number,model_path, input_shape, nb_classes, dataset, sens_params, update_list ):
        config = tf.ConfigProto()
        config.gpu_options.per_process_gpu_memory_fraction = 0.8
        sess = tf.Session(config = config)
        x = tf.placeholder(tf.float32, shape = input_shape)
        y = tf.placeholder(tf.float32, shape = (None, nb_classes))
        model = dnn(input_shape, nb_classes)   

        preds = model(x)
        saver = tf.train.Saver()
        model_path = model_path + dataset + "/test.model"
        saver.restore(sess, model_path)
#         l1 = model.layers[0].fprop(np.array([samples[0]]).astype('float32'))
#         r1 = model.layers[1].fprop(l1,[[0,0,0]],[1000])
#         l2 = model.layers[2].fprop(r1)
#         r2 = model.layers[3].fprop(l2)
#         init_out = r2.eval(session=sess)
        feed_dic = {}
        for neuron in range(len(update_list)):
            time1 = time.time()              
            for layer in range(0,11,2):
                if layer == 0:
                    l = model.layers[layer].fprop(samples.astype('float32'))
                else:
                    l = model.layers[layer].fprop(r)                   
                if layer + 1 == (layer_number * 2) - 1:
                    indices = []
                    for instance in range(l.shape[0]):                       
                        indices.append([ instance, 0, neuron])       
                    updates = [ update_list[ neuron ] ] * l.shape[0]                    
                    r = model.layers[layer + 1].fprop(l , indices, updates)
                else:
                    r = model.layers[layer + 1].fprop(l)             
            feed_dic[neuron] = r
        all_probs = sess.run(feed_dic)
        out_dic={}
        for key in all_probs.keys():
            probs = np.array(all_probs[key]).reshape((90,2))[:,1:].reshape((90))
            clus = clustering(probs,samples, sens_params)
            out_dic[key]= [len(clus) - 1 ]     
        sess.close()
        tf.reset_default_graph()
        return out_dic

def get_rate(sess, model, model_path, input_shape, nb_classes,
              dataset, lay_name, layer_output):
        def get_distance(vec1, vec2, size):
            return abs(vec1 - vec2).sum() / size
        if  sess._closed:
#                 config = tf.ConfigProto()
#                 config.gpu_options.per_process_gpu_memory_fraction = 0.8
                config = tf.ConfigProto(device_count = {'GPU': 0})
                config.allow_soft_placement= True
                sess   = tf.Session(config = config)
                x      = tf.placeholder(tf.float32, shape = input_shape)
                y      = tf.placeholder(tf.float32, shape = (None, nb_classes))
                model  = dnn(input_shape, nb_classes)   
                preds  = model(x)
                saver  = tf.train.Saver()
                saver.restore(sess, model_path)    

        max_dis = 0
        epsillon = 10 ** -7
        num_samples = len(layer_output[lay_name])

        layer_ind = np.where(np.array(list(layer_output.keys())) == lay_name)[0][0]
        for ind in range(layer_ind):
            temp_dis = 0
            if 'ReLU' in np.array(list(layer_output.keys()))[ind]:
                layer_name = np.array(list(layer_output.keys()))[ind]
                layer_size  = len(layer_output[layer_name][0][0])
                distances = np.zeros((num_samples,num_samples))
                for i in combinations(range(num_samples),2):
                    distances[i[0],i[1]] = get_distance(layer_output[layer_name][i[0]],layer_output[layer_name][i[1]],layer_size)
                if distances.max()> max_dis:
                    max_dis = distances.max()
        distances = np.zeros((num_samples,num_samples))
        for i in combinations(range(num_samples),2):
            distances[i[0],i[1]] = get_distance(layer_output[lay_name][i[0]],layer_output[lay_name][i[1]],len(layer_output[lay_name][0][0]))
        cur_dis = distances.max()

        change_rate = (cur_dis - max_dis ) / (max_dis + epsillon)      
        return change_rate 
#-------------------------------------------
    
def dnn_fair_testing(dataset, sens_params, model_path):
    """
    
    The implementation of ADF
    :param dataset: the name of testing dataset
    :param sensitive_param: the index of sensitive feature
    :param model_path: the path of testing model
    :param cluster_num: the number of clusters to form as well as the number of
            centroids to generate
    :param max_global: the maximum number of samples for global search
    :param max_local: the maximum number of samples for local search
    :param max_iter: the maximum iteration of global perturbation
    """
    data = {"census":census_data, "credit":credit_data, "bank":bank_data, "compas":compas_data, 
            "default": default_data, "heart":heart_data}
    data_config = {"census":census, "credit":credit, "bank":bank, "compas":compas, "default":default,
                  "heart":heart}
    # prepare the testing data and model
    X, Y, input_shape, nb_classes = data[dataset]()
    tf.set_random_seed(1234)
    
    

#     config = tf.ConfigProto()
#     config.gpu_options.per_process_gpu_memory_fraction = 0.8
    config = tf.ConfigProto(device_count = {'GPU': 0})
    config.allow_soft_placement= True
    sess = tf.Session(config=config)
    x = tf.placeholder(tf.float32, shape=input_shape)
    y = tf.placeholder(tf.float32, shape=(None, nb_classes))
    model = dnn(input_shape, nb_classes)   

    preds = model(x)
    saver = tf.train.Saver()
    model_path = model_path + dataset + "/test.model"
    saver.restore(sess, model_path)

    sample_df=pd.read_csv('../results/census/local_samples.csv',header='infer')
    loc_samples=np.array(sample_df.drop(columns=['sample']))
    for sample in loc_samples:

        samples = m_instance( np.array([sample]) , sens_params, data_config[dataset])
        layer_output = layer_out(sess,model,np.array(samples).astype('float32'))  
        temp={}
        for layer in layer_output.keys():
            if 'ReLU' in layer:
                temp[layer]=get_rate(sess, model, model_path, input_shape, nb_classes,
                                      dataset,layer, layer_output)
        

    

#    zz=sess.run(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,'linear/kernel:0'))
    #input([n.name for n in tf.get_default_graph().as_graph_def().node])
#    current_graph = tf.compat.v1.get_default_graph()   

    
#     all_names = [var for var in tf.trainable_variables()]
#     for x in all_names:
#         print(x)
#         if x.name == 'linear/kernel:0':
#             print(x.eval(session=sess))
#             op=x.assign(np.zeros((13,64)))
#             sess.run(op)
#             print('-------------------')
#             print(x.numpy())
#             print(x.eval(session=sess))
#     input('dd')
    
    #loc_samples = np.load('../results/census/local_samples.npy')
#     input(pd.DataFrame(loc_samples).to_csv('../results/census/local_samples.csv'))
    sample_df=pd.read_csv('../results/census/local_samples.csv',header='infer')
    loc_samples=np.array(sample_df.drop(columns=['sample']))
    update_df=pd.read_csv('../results/census/dataset_layer2_out.csv')
    update_list_mean = update_df.mean()
    update_list_max = update_df.max()
    update_list_mode = update_df.mode()
#---------------------------------------    
#     Layer 2 output for analysis

#     test_dic={}
#     ind=0
#     global zz
#     for sample in loc_samples:
#         samples = m_instance( np.array([sample]) , sens_params, data_config[dataset])
#         l1 = model.layers[0].fprop(samples.astype('float32'))
#         r1 = model.layers[1].fprop(l1)
#         l2 = model.layers[2].fprop(r1)
#         r2 = model.layers[3].fprop(l2)
#         test_dic[ind]=r2
#         ind+=1
#     zz=sess.run(test_dic)
#     df=pd.DataFrame(np.zeros((1,32))).drop(index=0)
#     for key in zz.keys():
#         df=df.append(pd.DataFrame(zz[key].reshape((90,32))),ignore_index=True)
#     df.to_csv('../results/census/dataset_layer2_out.csv',index=False) 
#-------------------------------------------------
#     input()
#         r1 = model.layers[1].fprop(l1,[[0,0,0]],[1000])
#         l2 = model.layers[2].fprop(r1)
#         r2 = model.layers[3].fprop(l2)
#     df_layer_rate =pd.DataFrame(np.zeros((1,5)),columns=['ReLU1','ReLU3','ReLU5','ReLU7','ReLU9'])
#     for sample in loc_samples:

#         samples = m_instance( np.array([sample]) , sens_params, data_config[dataset])
#         layer_output = layer_out(sess,model,np.array(samples).astype('float32'))  
#         temp={}
#         for layer in layer_output.keys():
#             if 'ReLU' in layer:
#                 #print(layer, get_rate(layer, layer_output)) 
#                 temp[layer]=get_rate(layer, layer_output)
#                 #df_layer_rate[ layer ] = get_rate(layer, layer_output) 
#         df_layer_rate = df_layer_rate.append(temp,ignore_index=True)

#     #df_layer_rate.to_csv('../results/census/layer_locator.csv',index=False) 
   
    layer_number = 2
    column_list=['ReLU3 N0','ReLU3 N1','ReLU3 N2','ReLU3 N3',
                 'ReLU3 N4','ReLU3 N5','ReLU3 N6','ReLU3 N7',
                 'ReLU3 N8','ReLU3 N9','ReLU3 N10','ReLU3 N11',
                 'ReLU3 N12','ReLU3 N13','ReLU3 N14','ReLU3 N15',
                 'ReLU3 N16','ReLU3 N17','ReLU3 N18','ReLU3 N19',
                 'ReLU3 N20','ReLU3 N21','ReLU3 N22','ReLU3 N23',
                 'ReLU3 N24','ReLU3 N25','ReLU3 N26','ReLU3 N27',
                 'ReLU3 N28','ReLU3 N29','ReLU3 N30','ReLU3 N31']
    df=pd.DataFrame(np.zeros((1,32)),dtype='int32').drop(index=0)
    for sample in loc_samples:
        samples = m_instance( np.array([sample]) , sens_params, data_config[dataset])
        ss=neuron_locator(samples, layer_number,model_path,input_shape,nb_classes,dataset,sens_params,update_list_max)
        df1 = pd.DataFrame(ss)
        df=df.append(df1,ignore_index=False)
    df.columns=column_list
    df.to_csv('../results/census/neuron_locator_max.csv',index=False)   
#----------------------------
                
           
        
def main(argv=None):
    dnn_fair_testing(dataset = FLAGS.dataset, 
                     sens_params = FLAGS.sens_params,
                     model_path  = FLAGS.model_path)


if __name__ == '__main__':
    flags.DEFINE_string("dataset", "census", "the name of dataset")
    flags.DEFINE_string('model_path', '../models/', 'the path for testing model')
    flags.DEFINE_integer('cluster_num', 4, 'the number of clusters to form as well as the number of centroids to generate')
    flags.DEFINE_list('sens_params',[9,8,1],'sensitive parameters index.1 for age, 9 for gender, 8 for race')
    tf.app.run()


INFO:tensorflow:Restoring parameters from ../models/census/test.model


I0722 11:40:30.492418 139931636864832 saver.py:1399] Restoring parameters from ../models/census/test.model
